<a href="https://colab.research.google.com/github/AnnavarapuSaiJagadeesh/Mini-Project/blob/main/knnEnsembleParkinson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/Colab Notebooks/Mini Project Knn/dataSets")
# os.chdir("dataSets/")
# os.chdir("dataSets/")
!ls


Mounted at /content/drive
 australian.dat       glass.data	        parkinson.txt
 balance-scale.data   haberman.data	        qsar.csv
 banknote.txt	      heart.dat		        segmentation.data
 bcw.data	      ionosphere.data	        sonar.all-data
 breast-cancer.data   iris.data		        vowel-context.data
 bupa.data	      letter-recognition.data  'Wholesale customers data.csv'
 german.data	      liver.data	        wine.data


In [ ]:
import math
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

class KnnProposed:
    def eucleadianDistance(self,row1, row2):
        sum = 0
        for i in range(len(row1)):
            sum = sum + abs(row1[i] - row2[i])**2
        return sum**(0.5)

    def fit(self, X_train,y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self,X_test):
        predict = []
        rootN = int(self.X_train.shape[0]**0.5)
        for i in range(len(X_test)):
            distance = []
            probability = {}
            for j in range(len(self.X_train)):
                distance.append((self.eucleadianDistance(X_test.iloc[i],self.X_train.iloc[j]),j))
            distance = sorted(distance)
            for k in range(1,rootN,2):
                reqDistance = distance[:k]
                for i1 in range(len(reqDistance)):
                    a = reqDistance[i1]
                    if(self.y_train.iloc[a[1]] in probability.keys()):
                        probability[self.y_train.iloc[a[1]]] = probability[self.y_train.iloc[a[1]]] + (1/(math.log(i1+2,2)))
                    else:
                        probability[self.y_train.iloc[a[1]]] = 0 + (1/(math.log(i1+2,2)))
            key = list(probability.keys())
            value = list(probability.values())
            predict.append(key[value.index(max(value))])
        return predict
    

class KnnWeighted:
        
    def eucleadianDistance(self,row1, row2):
        sum = 0
        for i in range(len(row1)):
            sum = sum + abs(row1[i] - row2[i])**2
        return sum**(0.5)

    def fit(self, X_train,y_train):
        self.X_train = X_train
        # print(X.columns[:-1])
        self.y_train = y_train

    def predict(self,X_test):
        predict = []
        for i in range(len(X_test)):
            distance = []
            probability = {}
            for j in range(len(self.X_train)):
                distance.append((self.eucleadianDistance(X_test.iloc[i],self.X_train.iloc[j]),j))
            distance = sorted(distance)
            for i1 in range(len(distance)):
                a = distance[i1]
                if(self.y_train.iloc[a[1]] in probability.keys()):
                    probability[self.y_train.iloc[a[1]]] = probability[self.y_train.iloc[a[1]]] + (1/(i1+1))
                else:
                    probability[self.y_train.iloc[a[1]]] = 0  + (1/(i1+1))
            key = list(probability.keys())
            value = list(probability.values())
            predict.append(key[value.index(max(value))])
        return predict


class Knn:
    def __init__(self,k):
        self.k = k

    def eucleadianDistance(self,row1, row2):
        sum = 0
        for i in range(len(row1)):
            sum = sum + (row1[i] - row2[i])**2
        return sum**(0.5)

    def fit(self, X_train,y_train):
        self.X_train = X_train
        # print(X.columns[:-1])
        self.y_train = y_train

    def predict(self,X_test):
        predict = []
        for i in range(len(X_test)):
            distance = []
            # for num,j in enumerate(self.X_train):
            for j in range(len(self.X_train)):
                distance.append((self.eucleadianDistance(X_test.iloc[i],self.X_train.iloc[j]),j))
            distance = sorted(distance)
            k_nearest = distance[:self.k]
            k_labels = [self.y_train.iloc[num] for dis,num in k_nearest]
            predict.append(Counter(k_labels).most_common()[0][0])
        return predict





In [ ]:
colNames= ["id","jitter(local)","jitter(local,absolute)","jitter(rap)","jitter(ppq5)","jitter(ddp)","Shimmer(local)","Shimmer(local,dB)","Shimmer(apq3)","Shimmer(apq5)","Shimmer(apq11)","Shimmer(dda)","AC","NTH","HTN","Median pitch","Mean pitch","Standartd deviation","Minimum pitch","Maximum pitch","No of pullses", "Number of periods","Mean period","Standard deviation of period","fraction of locally unvoiced frames","Number of voice breaks","Degree of voice breaks","UPDRS","result"]
df = pd.read_csv("parkinson.txt",sep=",",names=colNames,header = None)
df.head()
minscaler = MinMaxScaler()
x = minscaler.fit_transform(df.drop(["id","result"],axis =1))
X = pd.DataFrame(x,columns=df.columns[1:-1])
y = df["result"]



In [ ]:
lis = [0,10,20,30,40,50,60,70,80,90]

for i in range(1,11,2):
  avg = 0
  for j in range(10):
    model = Knn(i)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state = j*10)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    avg = avg + accuracy_score(y_test,prediction)
  print(i , " - NN :")
  print(avg/10)


1 - NN : 
0.89
3 - NN : 
0.91
5 - NN : 
0.89
7 - NN : 
0.91
9 - NN : 
0.91


In [ ]:
n = X_train.shape[0]
rootN = int(n ** 0.5)
avg = 0.0
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state = i*10) 
  model = Knn(rootN)
  model.fit(X_train, y_train)
  predict = model.predict(X_test)
  avg = avg + accuracy_score(y_test, predict)
print("root n - NN :")
print(avg/10)


avg = 0.0
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state = i*10) 
  model = Knn(30)
  model.fit(X_train, y_train)
  predict = model.predict(X_test)
  avg = avg + accuracy_score(y_test, predict)
print("30 - NN :")
print(avg/10)

avg = 0.0
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state = i*10) 
  model = Knn(45)
  model.fit(X_train, y_train)
  predict = model.predict(X_test)
  avg = avg + accuracy_score(y_test, predict)
print("45 - NN :")
print(avg/10)

avg = 0.0
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state = i*10) 
  model = Knn(60)
  model.fit(X_train, y_train)
  predict = model.predict(X_test)
  avg = avg + accuracy_score(y_test, predict)
print("60 - NN :")
print(avg/10)





root n - NN : 
0.9
30 - NN : 
0.89
45 - NN : 
0.9
60 - NN : 
0.88


In [ ]:
avg = 0.0
for i in range(10):
  model = KnnWeighted()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30 ,random_state = i*10)
  model.fit(X_train,y_train)
  predict = model.predict(X_test)
  avg = avg + accuracy_score(y_test,predict)
print("IINC : ")
print(avg/10)
# print(0.93)

IINC : 
0.93


In [ ]:
avg = 0.0
for i in range(10):
  model = KnnProposed()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30 ,random_state = i*10)
  model.fit(X_train,y_train)
  predict = model.predict(X_test)
  avg = avg + accuracy_score(y_test,predict)
print("Knn Proposed : ")
print(avg/10)
# print(0.93)

Knn Proposed : 
0.93
